<a href="https://colab.research.google.com/github/prateekjoshi565/textrank_text_summarization/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arjunkaranam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Read the CSV file
import io
df = pd.read_csv('tennis_articles_v4.csv')

In [3]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [45]:
df['article_text'][0]

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same 

In [51]:
# split the the text in the articles into sentences
sentences = []
sentences.append(sent_tokenize(df['article_text'][0]))  

In [52]:
sentences

[['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
  "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
  'I think everyone knows this is my job here.',
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
  "I'm a pretty competitive girl.",
  "I say my hellos, but I'm not sending any players flowers as well.",
  "Uhm, I'm not really friendly or close to many players.",
  "I have not a lot of friends away from the courts.'",
  'When she said she is not really close to a lot of players, is that something strategic that she is doing?',
  "Is it different on the men's tour than the women's tour?",
  "'No, 

In [53]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [54]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [55]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arjunkaranam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [57]:
# function to remove stopwords
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [58]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [59]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [60]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [61]:
len(sentence_vectors)

17

The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [62]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [65]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [66]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [70]:
# Specify number of sentences to form the summary
sn = 8

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

I think just because you're in the same sport doesn't mean that you have to be friends with everyone just because you're categorized, you're a tennis player, so you're going to get along with tennis players.
I think everyone just thinks because we're tennis players we should be the greatest of friends.
When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
When she said she is not really close to a lot of players, is that something strategic that she is doing?
Uhm, I'm not really friendly or close to many players.
I have not a lot of friends away from the courts.'
The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.
I think every person has dif